In [1]:
import pandas as pd 
from matplotlib import pyplot as plt
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, Date, LargeBinary, create_engine, func, delete
from sqlalchemy.orm import declarative_base, relationship, sessionmaker, Session
from sqlalchemy.sql.expression import select
from sqlalchemy.dialects.sqlite import insert
from sqlalchemy.exc import SQLAlchemyError
from faker import Faker
import random
from IPython.display import clear_output
import datetime
from random import randrange
from datetime import timedelta
import time
from cryptography.fernet import Fernet
import hashlib

In [2]:
start = time.time()
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

d = datetime.datetime.now().date()
d_plus_two_w = d.replace(day=d.day + 14)
d_plus_five_y = d.replace(year=d.year + 5)

In [3]:
key = Fernet.generate_key()
f= Fernet(key)

def encrypt(data):
    return f.encrypt(data.encode('utf-8'))

def decrypt(data):
    return f.decrypt(data.decode('utf-8'))

In [4]:
vg_sales = pd.read_csv('vgsales.csv')
vg_sales.Year = vg_sales.Year.fillna(0)
vg_sales.Publisher = vg_sales.Publisher.fillna('Unknown')

In [5]:
Base = declarative_base()

class Platform(Base):
    __tablename__='platforms'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False, unique=True)

    game_version = relationship('GameVersion', back_populates= 'platform')


class Rank(Base):
    __tablename__='ranks'

    id = Column(Integer, primary_key=True)
    ranking = Column(Integer, nullable=False, unique=True)

    game_version = relationship('GameVersion', back_populates= 'rank')

class Year(Base):
    __tablename__ = 'years'

    id = Column(Integer, primary_key=True)
    year_date = Column(Integer, nullable=False, unique=True)

    game_version = relationship('GameVersion', back_populates='year')

class Publisher(Base):
    __tablename__ = 'publishers'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)

    game_version = relationship('GameVersion', back_populates='publisher')

class Genre(Base):
    __tablename__ = 'genres'

    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False, unique=True)

    game_version = relationship('GameVersion', back_populates='genre')

class Game(Base):
    __tablename__ = 'games'

    id = Column(Integer, primary_key=True)
    name = Column(String(150), nullable=False, unique=True)

    game_version = relationship('GameVersion', back_populates='game')

class GameVersion(Base):
    __tablename__ = 'game_versions'

    id = Column(Integer, primary_key=True)

    platform_id = Column(Integer, ForeignKey('platforms.id'), nullable=False)
    platform = relationship('Platform', back_populates= 'game_version')

    rank_id = Column(Integer, ForeignKey('ranks.id'), nullable=False)
    rank = relationship('Rank', back_populates= 'game_version')

    year_id = Column(Integer, ForeignKey('years.id'))
    year = relationship('Year', back_populates='game_version')

    publisher_id = Column(Integer, ForeignKey('publishers.id'))
    publisher = relationship ('Publisher', back_populates='game_version')

    game_id = Column(Integer, ForeignKey('games.id'), nullable=False)
    game = relationship('Game', back_populates='game_version')

    genre_id = Column(Integer, ForeignKey('genres.id'))
    genre = relationship('Genre', back_populates='game_version')

    sale = relationship('Sale', back_populates='game_version')

    transaction = relationship('Transaction', back_populates='game_version')

class Sale(Base):
    __tablename__ = 'sales'

    id = Column (Integer, primary_key=True)
    na_sales = Column(Integer)
    eu_sales = Column(Integer)
    jp_sales = Column(Integer)
    other_sales = Column(Integer)

    game_version_id = Column(Integer, ForeignKey('game_versions.id'), nullable=False)
    game_version = relationship('GameVersion', back_populates='sale')

class Region(Base):
    __tablename__ = 'regions'

    id = Column(Integer, primary_key=True)
    name = Column(String(30), nullable=False, unique=True)

    country = relationship('Country', back_populates='region') 

class Country(Base):
    __tablename__ = 'countries'

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(100), nullable=False, unique=True)

    region_id = Column(Integer, ForeignKey('regions.id'), nullable=False)
    region = relationship('Region', back_populates='country')

    city = relationship('City', back_populates='country')

class City(Base):
    __tablename__ = 'cities'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)

    country_id = Column(Integer, ForeignKey('countries.id'), nullable=False)
    country = relationship('Country', back_populates='city')

    post_code = relationship('PostCode', back_populates='city')

class PostCode(Base):
    __tablename__ = 'post_codes'

    id = Column(Integer, primary_key=True)
    code = Column(String(10), nullable=False, unique=True)

    city_id = Column(Integer, ForeignKey('cities.id'), nullable=False)
    city = relationship('City', back_populates='post_code')

    user = relationship('User', back_populates='post_code')

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(LargeBinary, nullable=False)
    email = Column(String(255), nullable=False, unique=True)
    phone_number = Column(LargeBinary, nullable=False, unique=True)
    address = Column(LargeBinary, nullable=False)
    keep_info = Column(Boolean, nullable=False)
    deleted_in = Column(Date, default=random_date(d, d_plus_two_w))

    post_code_id = Column(Integer, ForeignKey('post_codes.id'), nullable=False)
    post_code = relationship('PostCode', back_populates='user')

    transaction = relationship('Transaction', back_populates='user')

class Transaction(Base):
    __tablename__ = 'transactions'

    id = Column(Integer, primary_key=True)
    quantity = Column (Integer, nullable=False)

    game_version_id = Column(Integer, ForeignKey('game_versions.id'), nullable=False)
    game_version = relationship('GameVersion', back_populates='transaction')

    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship('User', back_populates='transaction')


In [6]:
class check_insert_data:
    def __init__(self, session):
        self.session = session

    def region(self, region_name):
        region_name_db = self.session.query(Region).filter_by(name = region_name).first()
        if not region_name_db:
            region_name_db = Region(name = region_name)
            self.session.add(region_name_db)
            self.session.flush()
        return region_name_db

    def country(self, region_name, country_name):
        region_name_db = self.region(region_name)
        country_name_db = self.session.query(Country).filter_by(name = country_name).first()
        if not country_name_db:
            country_name_db = Country(name = country_name, region = region_name_db)
            self.session.add(country_name_db)
            self.session.flush()
        return country_name_db


    def city(self, region_name, country_name, city_name):
        country_name_db = self.country(region_name, country_name)
        city_name_db = self.session.query(City).filter_by(name = city_name).first()
        if not city_name_db:
            city_name_db = City(name = city_name, country = country_name_db)
            self.session.add(city_name_db)
            self.session.flush()
        return city_name_db
    
    def post_code(self, region_name, country_name, city_name, post_code_value):
        city_name_db = self.city(region_name, country_name, city_name)
        post_code_db = self.session.query(PostCode).filter_by(code = post_code_value).first()
        if not post_code_db:
            post_code_db = PostCode(code = post_code_value, city = city_name_db)
            self.session.add(post_code_db)
            self.session.flush()
        return post_code_db
    
    def platform(self, value):
        platform = self.session.query(Platform).filter_by(name = value).first()
        if not platform:
            platform = Platform(name = value)
            self.session.add(platform)
            self.session.flush()
        return platform
    
    def rank(self, value):
        rank = self.session.query(Rank).filter_by(ranking = value).first()
        if not rank:
            rank = Rank(ranking = value)
            self.session.add(rank)
            self.session.flush()
        return rank
    
    def year(self, value):
        year = self.session.query(Year).filter_by(year_date = value).first()
        if not year:
            year = Year(year_date = value)
            self.session.add(year)
            self.session.flush()
        return year
    
    def publisher(self, value):
        publisher = self.session.query(Publisher).filter_by(name = value).first()
        if not publisher:
            publisher = Publisher(name = value)
            self.session.add(publisher)
            self.session.flush()
        return publisher
    
    def genre(self, value):
        genre = self.session.query(Genre).filter_by(name = value).first()
        if not genre:
            genre = Genre(name = value)
            self.session.add(genre)
            self.session.flush()
        return genre

    def game(self, value):
        game = self.session.query(Game).filter_by(name = value).first()
        if not game:
            game = Game(name = value)
            self.session.add(game)
            self.session.flush()
        return game

In [7]:
def add_commit_close_clear(session, added_list):
    session.rollback()
    session.add_all(added_list)
    session.commit()
    session.close()
    clear_output()

def bulk_commit(session, class_, list):
    session.rollback()
    try:
        session.bulk_insert_mappings(class_, list)
        session.commit()
        clear_output()
        end = 'Inserted'
    except Exception as e:
        clear_output()
        print(e)
        end = 'Insertion failed'
    return end

In [8]:
engine = create_engine("sqlite:///video_games_sales.db", echo=True)

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()
clear_output()

In [9]:
session.rollback()
users_list = []
fake = Faker('fr_FR')
for _ in range(2**6):
    name = encrypt(fake.name().lower().lower())
    email = hashlib.sha256(fake.email().lower().encode()).hexdigest()
    phone_number = encrypt(fake.phone_number())
    try:
        address, city_code = fake.address().lower().split('\n')
        address = encrypt(address)
        post_code, city_name = city_code.lower().split(' ', 1)
        country_name = fake.country().lower()
    except: 
        continue
    region_name = random.choice(['EU', 'US', 'JP', 'Other'])
    keep_info = random.choice([True, True, True, True, False])
    deleted_in = random_date(d, d_plus_five_y) if keep_info == True else random_date(d, d_plus_two_w)

    db = check_insert_data(session=session)
    post_code_db = db.post_code(region_name=region_name, country_name=country_name, city_name=city_name, post_code_value=post_code)
    
    users_list.append(User(name= name, email = email, phone_number = phone_number, address = address, post_code = post_code_db, keep_info=keep_info, deleted_in=deleted_in))
clear_output()
add_commit_close_clear(session, users_list)

In [10]:
year_list = [{'year_date':int(i)} for i in  sorted(vg_sales.Year[vg_sales.Year.notna()].unique().tolist())]
rank_list = [{'ranking':i} for i in sorted(vg_sales.Rank.unique().tolist())]
publisher_list = [{'name':i} for i in sorted(vg_sales.Publisher[vg_sales.Publisher.notna()].unique().tolist())]
platform_list = [{'name':i} for i in sorted(vg_sales.Platform.unique().tolist())]
genre_list = [{'name':i} for i in sorted(vg_sales.Genre.unique().tolist())]
game_list = [{'name':i} for i in sorted(vg_sales.Name.unique().tolist())]

classes = [Year, Rank, Publisher, Platform, Genre, Game]
lists = [year_list, rank_list, publisher_list, platform_list, genre_list, game_list]
output =[]
for i in range(6):
    try:
        session.rollback()
        session.bulk_insert_mappings(classes[i], lists[i])
        session.commit()
        output.append('Inserted')
    except Exception as e:
        output.append(f'Insertion failed due to {e}')
clear_output()
output

['Inserted', 'Inserted', 'Inserted', 'Inserted', 'Inserted', 'Inserted']

In [11]:
year_records = session.query(Year.id, Year.year_date).all()
year_to_id = {year_date:id for id, year_date in year_records}

rank_records = session.query(Rank.id, Rank.ranking).all()
rank_to_id = {ranking:id for id, ranking in rank_records}

publisher_records = session.query(Publisher.id, Publisher.name).all()
publisher_to_id = {name:id for id, name in publisher_records}

platform_records = session.query(Platform.id, Platform.name).all()
platform_to_id = {name:id for id, name in platform_records}

genre_records = session.query(Genre.id, Genre.name).all()
genre_to_id = {name:id for id, name in genre_records}

game_records = session.query(Game.id, Game.name).all()
game_to_id = {name:id for id, name in game_records}

game_version_list = [{'platform_id': platform_to_id[value.Platform], 'rank_id':rank_to_id[value.Rank], 'year_id':year_to_id[value.Year], 'publisher_id':publisher_to_id[value.Publisher], 'game_id':game_to_id[value.Name], 'genre_id':genre_to_id[value.Genre]} for _, value in vg_sales[['Platform', 'Rank', 'Year', 'Publisher', 'Name', 'Genre']].iterrows()]
try:
    session.rollback()
    bulk_commit(session, GameVersion, game_version_list)
    session.commit()
    output.append('Inserted')
except Exception as e:
    output.append(f'Insertion failed due to {e}')

In [12]:
game_version_records = session.query(GameVersion.id.label('id'),Platform.name.label('platform'), Rank.ranking.label('rank'), Year.year_date.label('year_date'), Publisher.name.label('publisher'), Game.name.label('game'), GameVersion.id.label('game_version_id'),  Genre.name.label('genre')).join(GameVersion.platform).join(GameVersion.rank).join(GameVersion.year).join(GameVersion.publisher).join(GameVersion.game).join(GameVersion.genre).all()
records_data = [{'game_version_id': value.game_version_id,'Rank': value.rank, 'Platform': value.platform, 'Year_Date': value.year_date, 'Publisher': value.publisher, 'Game': value.game, 'Genre': value.genre} for value in game_version_records]
records_df = pd.DataFrame(records_data)
vg_sales2 = vg_sales.rename(columns={'Name':'Game', 'Year': 'Year_Date'}).copy()
match_columns = ['Rank', 'Platform', 'Year_Date', 'Publisher', 'Game', 'Genre']
merged_df = pd.merge(
    left=records_df,
    right=vg_sales2[['Rank', 'Platform', 'Year_Date', 'Publisher', 'Game', 'Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']],
    left_on=match_columns,
    right_on=match_columns,
    how='right'
)
sales_list = [{'game_version_id': value.game_version_id, 'na_sales':value.NA_Sales, 'eu_sales':value.EU_Sales, 'jp_sales':value.JP_Sales, 'other_sales':value.Other_Sales} for _, value in merged_df.iterrows()]
bulk_commit(session, Sale, sales_list)

'Inserted'

In [13]:
transaction_list = []
for i in range(256):
    quantity = int(random.normalvariate(1.5, 1)) + 1
    random_game_version = session.query(GameVersion.id).order_by(func.random()).first()
    random_user = session.query(User.id).order_by(func.random()).first()
    transaction_list.append({'quantity':quantity, 'game_version_id':random_game_version.id, 'user_id':random_user.id})
bulk_commit(session, Transaction, transaction_list)

'Inserted'

In [14]:
session.execute(delete(User).where(User.deleted_in <= datetime.datetime.now().date()))
session.commit()

2025-12-02 17:12:52,615 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 17:12:52,617 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.deleted_in <= ?
2025-12-02 17:12:52,618 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('2025-12-02',)
2025-12-02 17:12:52,619 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
end = time.time()
print('Injection time:', end - start)

Injection time: 3.9246158599853516


# BREAK

In [16]:
def insert_or_get_id(session: Session, class_, column_names: list[str], values: list):

    conflict_column = column_names[0] 
    conflict_value = values[0]
    
    try:
        
        session.rollback()
        values = list(map(lambda x: x.lower() if x is type(str) else x,values))
        values_to_insert = dict(zip(column_names, values))
        stmt = insert(class_).values(values_to_insert)
        on_conflict_stmt = stmt.on_conflict_do_nothing(
            index_elements=[conflict_column] 
        )
        returning_stmt = on_conflict_stmt.returning(class_.id)

        result = session.execute(returning_stmt)
        inserted_id = result.scalar_one_or_none()
        session.commit()

        if inserted_id is not None:
            return inserted_id
        
        else:
 
            column_attribute = getattr(class_, conflict_column) 
            
            existing_id = session.execute(
                select(class_.id).filter(column_attribute == conflict_value)
            ).scalar_one()
            
            print(f"ID existant trouvé : {existing_id}")
            return existing_id

    except SQLAlchemyError as e:
        session.rollback()
        print(f"❌ Erreur lors de l'exécution de la requête : {e}")
        return None

def add_user(name, email, phone_number, address, post_code, city, country, region_name, keep_info=False):
    session.rollback()
    encrypted_name = encrypt(name)
    encrypted_email = hashlib.sha256(email.encode()).hexdigest()
    encrypted_phone_number = encrypt(phone_number)
    encrypted_address = encrypt(address)
    deleted_in = d_plus_five_y if keep_info == True else d_plus_two_w

    column_names = ['name']
    values = [region_name]
    region_id = insert_or_get_id(session, Region, column_names, values)

    column_names = ['name', 'region_id']
    values = [country, region_id]
    country_id = insert_or_get_id(session, Country, column_names, values)

    column_names = ['name', 'country_id']
    values = [city, country_id]
    city_id = insert_or_get_id(session, City, column_names, values)

    column_names = ['code', 'city_id']
    values = [post_code, city_id]
    post_code_id = insert_or_get_id(session, PostCode, column_names, values)
    post_code_object = session.execute(
        select(PostCode).where(PostCode.id == post_code_id)
    ).scalar_one()

    new_user = User(name = encrypted_name, email=encrypted_email, phone_number=encrypted_phone_number, address=encrypted_address, keep_info=keep_info, deleted_in=deleted_in, post_code = post_code_object)
    session.rollback()
    session.add(new_user)
    session.commit()

In [17]:
session.rollback()
new_user = add_user('Cyril Leconte', 'cyril.leconte@email.com', '+33 6 06 06 06 06', '1 champ elysee', '75000', 'Paris', 'France', 'EU')
new_user

2025-12-02 17:12:52,665 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 17:12:52,666 INFO sqlalchemy.engine.Engine INSERT INTO regions (name) VALUES (?) ON CONFLICT (name) DO NOTHING RETURNING id
2025-12-02 17:12:52,666 INFO sqlalchemy.engine.Engine [no key 0.00033s] ('EU',)
2025-12-02 17:12:52,667 INFO sqlalchemy.engine.Engine COMMIT
2025-12-02 17:12:52,668 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 17:12:52,669 INFO sqlalchemy.engine.Engine SELECT regions.id 
FROM regions 
WHERE regions.name = ?
2025-12-02 17:12:52,670 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ('EU',)
ID existant trouvé : 3
2025-12-02 17:12:52,671 INFO sqlalchemy.engine.Engine ROLLBACK
2025-12-02 17:12:52,672 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 17:12:52,672 INFO sqlalchemy.engine.Engine INSERT INTO countries (name, region_id) VALUES (?, ?) ON CONFLICT (name) DO NOTHING RETURNING id
2025-12-02 17:12:52,673 INFO sqlalchemy.engine.Engine [no key 0.00044s] ('Fran

In [18]:
def retrive_user_with_mail(mail):
    encrypted_email = hashlib.sha256(mail.encode()).hexdigest()
    stmt = select(User).where(User.email == encrypted_email)
    user = session.execute(stmt).scalar_one_or_none()
    return user
user = retrive_user_with_mail('cyril.leconte@email.com')
decrypt(user.name).decode()

2025-12-02 17:12:52,699 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-02 17:12:52,701 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email, users.phone_number, users.address, users.keep_info, users.deleted_in, users.post_code_id 
FROM users 
WHERE users.email = ?
2025-12-02 17:12:52,701 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ('44d73947dc9aa54153a1e2b7756b5c8eac7ec4fb149054a4fb18576a603b9e6e',)


'Cyril Leconte'